# Step 3-3 Multivariate DL models

Multivariate Deep Learning models.

The models are multivariate versions from the baseline models: LSTM, Bi-directional LSTM, ED-LSTM, CNN

**Data prep and train workflow**:

raw data -> split train and test -> pct_change normalisation -> combine all countries' train data -> fit universal StandardScaler

-> create sequences (5 input windows, 1 output window) -> train models with hyperparmeter tuning

**Test and evaluation workflow**:

for each country, combine train + test data (lags) -> calculate pct_change -> scale with the previous Scaler -> create sequences

-> Extract the last 9 sequences -> Model prediction -> Inverse StandardScaler -> Denormalise -> calculate metrics

The selected features from the previous feature selection step:

* Key features = `gdp`, `primary_energy_consumption`, `population`

* Selected features = `oil_production`, `nulcear_consumption`, `wind_consumption`, `biofuel_consumption`, `energy_per_gdp`



### Necessary imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Conv1D, Flatten
from keras.layers import RepeatVector, TimeDistributed, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

import warnings
import os

warnings.filterwarnings("ignore")

### Config

In [4]:
TARGET_VARIABLES = 'co2'
SELECTED_COUNTRIES = ['United States', 'China', 'India']
MAX_LAGS = 4
N_STEPS_IN = 5
N_STEPS_OUT = 1
TEST_SIZE = 9
save_dir = 'data/03_03_results'
os.makedirs(save_dir, exist_ok=True)

MULTIVARIATE_FEATURES = [
    'gdp',
    'primary_energy_consumption',
    'population',
    'oil_production',
    'nuclear_consumption',
    'wind_consumption',
    'biofuel_consumption',
    'energy_per_gdp'
]

ARIMA_ORDERS = {
    'United States': (0, 1, 0),
    'China': (0, 2, 0),
    'India': (1, 1, 1)
}

### Hyperparameter Tuning Config

In [5]:
DL_CONFIGS = {
    'LSTM': [
        {'hidden': 16, 'epochs': 100, 'dropout': 0.0},
        {'hidden': 32, 'epochs': 100, 'dropout': 0.0},
        {'hidden': 64, 'epochs': 100, 'dropout': 0.2}
    ],
    'Bi-LSTM': [
        {'hidden': 8, 'epochs': 100, 'dropout': 0.0},
        {'hidden': 16, 'epochs': 100, 'dropout': 0.0},
        {'hidden': 32, 'epochs': 100, 'dropout': 0.2}
    ],
    'ED-LSTM': [
        {'hidden': 8, 'epochs': 100, 'dropout': 0.0},
        {'hidden': 16, 'epochs': 100, 'dropout': 0.0},
        {'hidden': 32, 'epochs': 100, 'dropout': 0.2}
    ],
    'CNN': [
        {'filters': 16, 'epochs': 100},
        {'filters': 32, 'epochs': 100},
        {'filters': 64, 'epochs': 100}
    ]
}

### Data Load

In [6]:
def load_data(save_dir='data'):
    data_files = {
        'all_data_df': os.path.join(save_dir, 'all_data_df.csv'),
        'g20_lag_df': os.path.join(save_dir, 'g20_lag_df.csv'),
        'lag_three_sel_1969_df': os.path.join(save_dir, 'lag_three_sel_1969_df.csv')
    }

    dfs = {}
    for name, filepath in data_files.items():
        if os.path.exists(filepath):
            dfs[name] = pd.read_csv(filepath)
            print(f"Loaded {name}: {dfs[name].shape}")
        else:
            print(f"{filepath} not found")
    
    return dfs

In [7]:
data = load_data()
all_data_df = data['all_data_df']
g20_lag_df = data['g20_lag_df']
g20_lag_1969_df = g20_lag_df[g20_lag_df['year'] >= 1969].copy()
g20_lag_1969_df = g20_lag_1969_df[g20_lag_1969_df['year'] < 2023]
lag_three_sel_1969_df = data['lag_three_sel_1969_df']

Loaded all_data_df: (55529, 200)
Loaded g20_lag_df: (3744, 992)
Loaded lag_three_sel_1969_df: (162, 992)


### Data Prep

In [8]:
def tts_by_year(df, test_size=9):
    train_data = {}
    test_data = {}

    for country in df['country'].unique():
        country_data = df[df['country'] == country].sort_values('year')

        split_idx = len(country_data) - test_size
        train_data[country] = country_data.iloc[:split_idx]
        test_data[country] = country_data.iloc[split_idx:]

    train_df = pd.concat(train_data.values(), ignore_index=True)
    test_df = pd.concat(test_data.values(), ignore_index=True)

    return train_df, test_df

In [9]:
train_3_df, test_3_df = tts_by_year(lag_three_sel_1969_df, TEST_SIZE)
train_g20_df, test_g20_df = tts_by_year(g20_lag_1969_df, TEST_SIZE)

print(f"Train data shape: {train_3_df.shape}")
print(f"Test data shape: {test_3_df.shape}")

Train data shape: (135, 992)
Test data shape: (27, 992)


### Helper functions

In [10]:
def mase(y_actual, y_pred, period=1):
    mae_forecast = mean_absolute_error(y_actual, y_pred)

    naive_forecast = y_actual[:-period] if period > 0 else y_actual[:-1]
    actual_for_naive = y_actual[period:] if period > 0 else y_actual[1:]

    if len(naive_forecast) == 0:
        return np.nan
    
    mae_naive = mean_absolute_error(actual_for_naive, naive_forecast)

    if mae_naive == 0:
        return 0 if mae_forecast == 0 else np.inf
    
    return mae_forecast / mae_naive

In [11]:
def calculate_pct_change(df, features, max_lags=MAX_LAGS):
    df_copy = df.copy()
    df_copy = df_copy.sort_values(['country', 'year']).reset_index(drop=True)
    pct_change_cols = []

    for feature in features:
        if feature not in df_copy.columns:
            continue

        # Pct change on current values
        lag1_col = f"{feature}_lag1"
        if lag1_col in df_copy.columns:
            df_copy[f"{feature}_pct_change"] = ((df_copy[feature] - df_copy[lag1_col]) / df_copy[lag1_col] * 100)
            pct_change_cols.append(f"{feature}_pct_change")

        # Pct change on lagged values
        for lag in range(1, max_lags):
            lag_col = f"{feature}_lag{lag}"
            prev_lag_col = f"{feature}_lag{lag+1}"
            
            if lag_col in df_copy.columns and prev_lag_col in df_copy.columns:
                df_copy[f"{lag_col}_pct_change"] = ((df_copy[lag_col] - df_copy[prev_lag_col]) / df_copy[prev_lag_col] * 100)
                pct_change_cols.append(f"{lag_col}_pct_change")

        # Lag4 for the first row = 0, then shift lag3_pct by country
        last_lag_col = f"{feature}_lag{max_lags}"
        lag3_pct_col = f"{feature}_lag{max_lags-1}_pct_change"

        if last_lag_col in df_copy.columns and lag3_pct_col in df_copy.columns:
            df_copy[f"{last_lag_col}_pct_change"] = df_copy.groupby('country')[lag3_pct_col].shift(1).fillna(0)
            pct_change_cols.append(f"{last_lag_col}_pct_change")
            
        df_copy = df_copy.replace([np.inf, -np.inf], np.nan)
    
    return df_copy, pct_change_cols

In [12]:
def create_sequences(data, n_steps_in, n_features):
    X, y = [], []

    for i in range(len(data) - n_steps_in):
        X.append(data[i:i + n_steps_in])
        y.append(data[i + n_steps_in, 0]) # Target co2 is the first column

    return np.array(X), np.array(y)

### Model Builds

In [13]:
def build_lstm(input_shape, hidden=16, dropout=0.0):
    model = Sequential([
        LSTM(hidden, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Dropout(dropout),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

In [14]:
def build_bilstm(input_shape, hidden=8, dropout=0.0):
    model = Sequential([
        Bidirectional(LSTM(hidden, activation='relu', kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.01)),
                        input_shape=input_shape),
        Dropout(dropout),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

In [15]:
def build_edlstm(input_shape, hidden=8, dropout=0.0):
    model = Sequential([
        LSTM(hidden, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Dropout(dropout),
        RepeatVector(N_STEPS_OUT),
        LSTM(hidden, activation='relu', return_sequences=True, kernel_regularizer=l2(0.01)),
        TimeDistributed(Dense(1))
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

In [16]:
def build_cnn(input_shape, filters=16):
    model = Sequential([
        Conv1D(filters=filters, kernel_size=2, activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=l2(0.01)),
        Flatten(),
        Dense(8, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

### Model Train

In [17]:
# Data Prep
def prepare_train_data(train_df, features, target):
    all_train_pct_data = []
    country_train_info = {}

    for country in train_df['country'].unique():
        country_data = train_df[train_df['country'] == country].sort_values('year').reset_index(drop=True)

        # Caculate pct_change for all features
        country_pct, pct_cols = calculate_pct_change(country_data, features, MAX_LAGS)

        # pct_change values for all features
        feature_pct_values = []
        for feat in features:
            pct_col = f"{feat}_pct_change"
            if pct_col in country_pct.columns:
                pct_values = country_pct[pct_col].fillna(0).values
                feature_pct_values.append(pct_values)
            else:
                print(f"{pct_col} not in {country}")
        
        # Stacking features: shape (timesteps, n_features)
        feature_pct_array = np.column_stack(feature_pct_values)
        all_train_pct_data.append(feature_pct_array)
        
        country_train_info[country] = {
            'pct_values': feature_pct_array,
            'original_values': country_data[target].values,
            'years': country_data['year'].values
        }
        
        print(f"{country} pct_values shape: {feature_pct_array.shape}")
    
    return all_train_pct_data, country_train_info

In [18]:
all_train_pct_data, country_train_info = prepare_train_data(train_3_df, MULTIVARIATE_FEATURES, TARGET_VARIABLES)

China pct_values shape: (45, 8)
India pct_values shape: (45, 8)
United States pct_values shape: (45, 8)


In [19]:
# Fit universal StandardScaler
def fit_scaler(all_train_pct_data, features):
    all_train_pct_combined = np.vstack(all_train_pct_data)
    print(f"Combined train data shape: {all_train_pct_combined.shape}")

    scaler = StandardScaler()
    scaler.fit(all_train_pct_combined)

    for i, feat in enumerate(features):
        print(f"{feat} mean={scaler.mean_[i]:.4f}, std={np.sqrt(scaler.var_[i]):.4f}")

    return scaler

In [20]:
scaler = fit_scaler(all_train_pct_data, MULTIVARIATE_FEATURES)

Combined train data shape: (135, 8)
gdp mean=4.8601, std=3.1878
primary_energy_consumption mean=4.5367, std=4.6948
population mean=1.4430, std=0.5910
oil_production mean=3.7511, std=9.7537
nuclear_consumption mean=15.2010, std=72.4845
wind_consumption mean=2225.8822, std=25395.8016
biofuel_consumption mean=7.5562, std=22.1054
energy_per_gdp mean=-0.2989, std=3.5189


In [21]:
def create_train_sequences(country_train_info, scaler, features):
    X_train_all = []
    y_train_all = []

    for country, info in country_train_info.items():
        pct_values = info['pct_values']
        scaled_values = scaler.transform(pct_values)

        X_country, y_country = create_sequences(scaled_values, N_STEPS_IN, len(features))

        X_train_all.append(X_country)
        y_train_all.append(y_country)

        print(f"{country} X_train shape: {X_country.shape}, y_train shape: {y_country.shape}")
    
    X_train = np.vstack(X_train_all)
    y_train = np.concatenate(y_train_all)
    
    print(f"\nCombined training sequences:")
    print(f"    X_train shape: {X_train.shape}")
    print(f"    y_train shape: {y_train.shape}")
    
    return X_train, y_train

In [22]:
X_train, y_train = create_train_sequences(country_train_info, scaler, MULTIVARIATE_FEATURES)

China X_train shape: (40, 5, 8), y_train shape: (40,)
India X_train shape: (40, 5, 8), y_train shape: (40,)
United States X_train shape: (40, 5, 8), y_train shape: (40,)

Combined training sequences:
    X_train shape: (120, 5, 8)
    y_train shape: (120,)


In [23]:
# Train model and Tune hyperparameters
def train_model_and_tuning(X_train, y_train):
    dl_models = {
        'LSTM': build_lstm,
        'Bi-LSTM': build_bilstm,
        'ED-LSTM': build_edlstm,
        'CNN': build_cnn
    }

    trained_models = {}

    for model_name, model_func in dl_models.items():
        print(f"\nTraining {model_name}")

        best_val_loss = np.inf
        best_model = None
        best_config = None

        # Reshape y_train for ED-LSTM output shape (n_samples, n_steps_out, n_features)
        if model_name == 'ED-LSTM':
            y_train_model = y_train.reshape(-1, N_STEPS_OUT, 1)
            print(f"ED-LSTM target shape: {y_train_model.shape}")
        else:
            y_train_model = y_train
            print(f"{model_name} target shape: {y_train_model.shape}")

        for config in DL_CONFIGS[model_name]:
            epochs = config['epochs']
            config_params = {k: v for k, v in config.items() if k != 'epochs'}

            input_shape = (X_train.shape[1], X_train.shape[2])

            if model_name == 'CNN':
                model = model_func(input_shape, filters=config_params['filters'])
            else:
                model = model_func(input_shape, hidden=config_params['hidden'], dropout=config_params.get('dropout', 0.0))

            history = model.fit(X_train, y_train_model, epochs=epochs, batch_size=16, validation_split=0.1, verbose=0)

            val_loss = history.history['val_loss'][-1]
            print(f"    For config: {config}: val_loss = {val_loss:.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                best_config = config
            else:
                tf.keras.backend.clear_session()
                del model
        
        trained_models[model_name] = {
            'model': best_model,
            'config': best_config
        }
        
        print(f"\nBest {model_name} - config: {best_config}, val_loss: {best_val_loss:.4f}")
    
    return trained_models

In [24]:
trained_models = train_model_and_tuning(X_train, y_train)


Training LSTM
LSTM target shape: (120,)


2025-11-04 19:19:25.206703: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-11-04 19:19:25.206747: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-11-04 19:19:25.206754: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2025-11-04 19:19:25.206785: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-04 19:19:25.206795: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-11-04 19:19:25.568291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


    For config: {'hidden': 16, 'epochs': 100, 'dropout': 0.0}: val_loss = 0.8060
    For config: {'hidden': 32, 'epochs': 100, 'dropout': 0.0}: val_loss = 1.4612
    For config: {'hidden': 64, 'epochs': 100, 'dropout': 0.2}: val_loss = 1.1265

Best LSTM - config: {'hidden': 16, 'epochs': 100, 'dropout': 0.0}, val_loss: 0.8060

Training Bi-LSTM
Bi-LSTM target shape: (120,)
    For config: {'hidden': 8, 'epochs': 100, 'dropout': 0.0}: val_loss = 0.6709
    For config: {'hidden': 16, 'epochs': 100, 'dropout': 0.0}: val_loss = 0.7458
    For config: {'hidden': 32, 'epochs': 100, 'dropout': 0.2}: val_loss = 1.1401

Best Bi-LSTM - config: {'hidden': 8, 'epochs': 100, 'dropout': 0.0}, val_loss: 0.6709

Training ED-LSTM
ED-LSTM target shape: (120, 1, 1)
    For config: {'hidden': 8, 'epochs': 100, 'dropout': 0.0}: val_loss = 0.5623
    For config: {'hidden': 16, 'epochs': 100, 'dropout': 0.0}: val_loss = 0.7009
    For config: {'hidden': 32, 'epochs': 100, 'dropout': 0.2}: val_loss = 0.8419

B

In [27]:
# Evaluation
def eval_on_test(train_df, test_df, target, features, scaler, trained_models, selected_countries):
    all_results = {}

    for country in selected_countries:
        print(f"{country.upper()}")

        train_country = train_df[train_df['country'] == country].sort_values('year').reset_index(drop=True)
        test_country = test_df[test_df['country'] == country].sort_values('year').reset_index(drop=True)

        combined_data = pd.concat([train_country, test_country], ignore_index=True)

        # Calculating pct_change for combined data
        combined_pct, _ = calculate_pct_change(combined_data, features, MAX_LAGS)

        # Stack pct_change values for all features
        feature_pct_values = []
        for feat in features:
            pct_col = f"{feat}_pct_change"
            if pct_col in combined_pct.columns:
                pct_values = combined_pct[pct_col].fillna(0).values
                feature_pct_values.append(pct_values)

        stacked_combined_pct = np.column_stack(feature_pct_values)
        
        # Fit the StandardScaler used before
        combined_scaled = scaler.transform(stacked_combined_pct)

        # Original values for calculating errors
        original_values = combined_data[target].values

        X_all, y_all = create_sequences(combined_scaled, N_STEPS_IN, len(features))

        X_test = X_all[-TEST_SIZE:]
        test_years = test_country['year'].values

        print(f"Test sequence shape: {X_test.shape}")

        country_results = {}

        for model_name, model_info in trained_models.items():
            model = model_info['model']
            config = model_info['config']

            # The model predicts scaled values due to the scaled train data
            preds_scaled = model.predict(X_test, verbose=0)

            # Flatten for ED-LSTM 3D output (n_samples, n_steps_out, features)
            if len(preds_scaled.shape) == 3:
                preds_scaled = preds_scaled[:, 0, 0]
            else:
                # Others flatten with ravel
                preds_scaled = preds_scaled.ravel()

            # Inverse transform co2 (first feature)
            tmp_array = np.zeros((len(preds_scaled), len(features)))
            tmp_array[:, 0] = preds_scaled
            preds_pct_inverse = scaler.inverse_transform(tmp_array)
            preds_pct_change = preds_pct_inverse[:, 0]

            # Denormalise pct change to forecasted actual CO2 values
            forecast = []
            test_start_idx = len(original_values) - TEST_SIZE

            for i in range(TEST_SIZE):
                prev_value = original_values[test_start_idx + i - 1]
                predicted_value = prev_value * (1 + preds_pct_change[i] / 100)
                forecast.append(predicted_value)

            forecast = np.array(forecast)
            actual_test = original_values[-TEST_SIZE:]
            
            # Calculate metrics
            individual_errors = np.abs(actual_test - forecast)
            rmse_score = np.sqrt(mean_squared_error(actual_test, forecast))
            mase_score = mase(actual_test, forecast)

            country_results[model_name] = {
                'forecast': forecast,
                'actual': actual_test,
                'individual_errors': individual_errors,
                'test_years': test_years,
                'RMSE': rmse_score,
                'MASE': mase_score,
                'best_config': config
            }

            print(f"\n  {model_name}")
            print(f"    RMSE: {rmse_score:.4f}")
            print(f"    MASE: {mase_score:.4f}")
            print(f"    Config: {config}")
        
        all_results[country] = country_results
    
    return all_results

In [28]:
dl_results = eval_on_test(train_3_df, test_3_df, TARGET_VARIABLES, MULTIVARIATE_FEATURES,
                          scaler, trained_models, SELECTED_COUNTRIES)

UNITED STATES
Test sequence shape: (9, 5, 8)

  LSTM
    RMSE: 376.5223
    MASE: 1.6369
    Config: {'hidden': 16, 'epochs': 100, 'dropout': 0.0}

  Bi-LSTM
    RMSE: 413.9216
    MASE: 1.9449
    Config: {'hidden': 8, 'epochs': 100, 'dropout': 0.0}

  ED-LSTM
    RMSE: 340.9380
    MASE: 1.4963
    Config: {'hidden': 8, 'epochs': 100, 'dropout': 0.0}

  CNN
    RMSE: 363.2374
    MASE: 1.5657
    Config: {'filters': 16, 'epochs': 100}
CHINA
Test sequence shape: (9, 5, 8)

  LSTM
    RMSE: 582.5533
    MASE: 1.9768
    Config: {'hidden': 16, 'epochs': 100, 'dropout': 0.0}

  Bi-LSTM
    RMSE: 636.3770
    MASE: 2.2832
    Config: {'hidden': 8, 'epochs': 100, 'dropout': 0.0}

  ED-LSTM
    RMSE: 656.0639
    MASE: 2.3311
    Config: {'hidden': 8, 'epochs': 100, 'dropout': 0.0}

  CNN
    RMSE: 658.6271
    MASE: 2.3431
    Config: {'filters': 16, 'epochs': 100}
INDIA
Test sequence shape: (9, 5, 8)

  LSTM
    RMSE: 145.8430
    MASE: 0.7527
    Config: {'hidden': 16, 'epochs': 100, 'dr

### Summary Table

In [29]:
summary_data = []

for country in SELECTED_COUNTRIES:
    for model_name, result in dl_results[country].items():
        summary_data.append({
            'Country': country,
            'Model': model_name,
            'RMSE': result['RMSE'],
            'MASE': result['MASE']
        })

summary_df = pd.DataFrame(summary_data)

# RMSE pivot table
rmse_pivot = summary_df.pivot(index='Model', columns='Country', values='RMSE')
rmse_pivot = rmse_pivot.round(4)

# Ordering models
model_order = ['LSTM', 'Bi-LSTM', 'ED-LSTM', 'CNN']
rmse_pivot = rmse_pivot.reindex([m for m in model_order if m in rmse_pivot.index])

print(rmse_pivot)

Country     China     India  United States
Model                                     
LSTM     582.5533  145.8430       376.5223
Bi-LSTM  636.3770  164.8177       413.9216
ED-LSTM  656.0639  152.8127       340.9380
CNN      658.6271  144.9349       363.2374


In [31]:
# MASE pivot table
mase_pivot = summary_df.pivot(index='Model', columns='Country', values='MASE')
mase_pivot = mase_pivot.round(4)

# Ordering models
model_order = ['LSTM', 'Bi-LSTM', 'ED-LSTM', 'CNN']
mase_pivot = mase_pivot.reindex([m for m in model_order if m in mase_pivot.index])

print(mase_pivot)

Country   China   India  United States
Model                                 
LSTM     1.9768  0.7527         1.6369
Bi-LSTM  2.2832  0.8586         1.9449
ED-LSTM  2.3311  0.7879         1.4963
CNN      2.3431  0.7981         1.5657


In [ ]:
pivot_table_filepath = os.path.join(save_dir, 'Multi_DL_summary.md')
with open(pivot_table_filepath, 'w') as f:
    f.write("# Summary of Multivariate DL models\n\n")
    f.write("RMSE of each DL model trained on all countries combined dataset with selected features for 3 countries\n\n")
    f.write(rmse_pivot.to_markdown())
    f.write("\n\n")

    f.write("---\n\n")

    f.write("MASE of each DL model trained on all countries combined dataset with selected features for 3 countries\n\n")
    f.write(mase_pivot.to_markdown())
    f.write("\n\n")

### Visualisation of forecasts